In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import json
import pickle
import logging

import pandas as pd
import numpy as np

from itext2kg import iText2KG
from itext2kg.utils import PubtatorProcessor
from langchain_ollama import ChatOllama, OllamaEmbeddings
from itext2kg.graph_integration import GraphIntegrator

/home/jovyan/miniconda3/envs/kg/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
URI = "neo4j+s://d7f45b5f.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "uQK8FUAR8QUDyeKsMLcGh4DcLeq306tq-5CGCZWmZvk"

llm = ChatOllama(
    model="deepseek-r1:32b",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [4]:
df = pd.read_csv("/home/jovyan/my_code/RAG/AD_pmid_results.csv")
pmid_list = list(df['pmid'].astype(str))
pmid_list = pmid_list[:10]

In [5]:
pmid_list

['39652311',
 '39890535',
 '39855320',
 '39589570',
 '39604278',
 '39596657',
 '39568323',
 '39657091',
 '39276711',
 '39431763']

In [6]:


#-------------PATH---------
PUTATOR_PATH = "/home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/"
JSON_PATH = "/home/jovyan/my_code/itext2kg/output_kg/AD"
OUTPUT_PATH = JSON_PATH
# 39890535

PMID = '39589570'  # 这个 PMID 提取出来全是散的节点，没有的关系，重新跑一下，看看问题在哪里了

In [7]:
with open(f"{PUTATOR_PATH}{PMID}.txt", "r") as f:
    text = f.readlines()

In [8]:
text

["39589570|t|Analysis of Alzheimer's disease associated deleterious non-synonymous single nucleotide polymorphisms and their impacts on protein structure and function by performing in-silico methods.\n",
 "39589570|a|Alzheimer's disease (AD) is a neurodegenerative disorder that is presented with a progressive loss of memory, a decline in cognitive abilities and multiple changes in behavior. Its pathogenicity has been linked to genetic factors in approximately 60-80% of the cases specifically APOE gene family and as well as other gene families. This study utilized advanced computational biology methods to analyze AD-associated nsSNPs extracted from the NHGRI-EBI GWAS Catalog. Ensembl Variant Effect Predictor (VEP) is used to annotate the variants associated with AD. Annotated missense variants were subjected to PolyPhen-2, SNPs&Go, PredictSNP servers which were used to predict pathogenicity of selected missense variants by protein sequence information. DynaMut and DUET servers were appl

In [9]:
pubtator_file = f"{PUTATOR_PATH}{PMID}.txt"
pubtator_process = PubtatorProcessor(pubtator_file, llm)
semantic_blocks = pubtator_process.block
properties_info = pubtator_process.properties_info
pubtator_info = pubtator_process.pubtator_info

2025-02-28 10:30:32,731 - INFO - Processing PubTator file: /home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/39589570.txt
2025-02-28 10:30:32,733 - WARNING - Skipping malformed entity line: ['39589570', 'Positive_Correlation', 'MESH:D000544', 'RS#:142412517;HGVS:p.R239W;CorrespondingGene:10452']
2025-02-28 10:30:32,734 - WARNING - Skipping malformed entity line: ['39589570', 'Positive_Correlation', 'MESH:D000544', 'RS#:34759087;HGVS:p.E987K;CorrespondingGene:3913']
2025-02-28 10:30:32,735 - WARNING - Skipping malformed entity line: ['39589570', 'Positive_Correlation', 'MESH:D000544', 'RS#:7412;HGVS:p.R176C;CorrespondingGene:348']
2025-02-28 10:30:32,736 - WARNING - Skipping malformed entity line: ['39589570', 'Positive_Correlation', 'MESH:D000544', 'RS#:2228467;HGVS:p.V41A;CorrespondingGene:1238']
2025-02-28 10:30:32,737 - WARNING - Skipping malformed entity line: ['39589570', 'Positive_Correlation', 'MESH:D000544', 'RS#:138799625;HGVS:p.G395S;CorrespondingGene:79895']
2025-02-28 10:30:32,7

2025-02-28 10:30:34,767 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-02-28 10:30:57,254 - INFO - Abstract distilled successfully.
2025-02-28 10:30:57,255 - INFO - Adding Alzheimer's disease to the abstract
2025-02-28 10:30:57,256 - INFO - Adding AD to the abstract
2025-02-28 10:30:57,257 - INFO - Adding neurodegenerative disorder to the abstract
2025-02-28 10:30:57,258 - INFO - Adding loss of memory to the abstract
2025-02-28 10:30:57,258 - INFO - Adding decline in cognitive abilities to the abstract
2025-02-28 10:30:57,259 - INFO - Adding APOE to the abstract
2025-02-28 10:30:57,259 - INFO - Adding ACKR2 to the abstract
2025-02-28 10:30:57,260 - INFO - Adding ATP8B4 to the abstract
2025-02-28 10:30:57,261 - INFO - Adding LAMB2 to the abstract
2025-02-28 10:30:57,261 - INFO - Adding TOMM40 to the abstract
2025-02-28 10:30:57,262 - INFO - Adding [{'proteinmutation': 'V41A'}, {'proteinmutation': 'R176C'}, {'proteinmutation': 'G395S'}, {'proteinmutati

['R239W'] {'proteinmutation': 'R239W'}


In [10]:
properties_info

{'proteinmutation': 'R239W', 'source': 'PMID39589570'}

In [11]:
semantic_blocks

['disease - [[\'disease\': "Alzheimer\'s disease"], [\'disease\': "Alzheimer\'s disease"], [\'disease\': \'AD\'], [\'disease\': \'neurodegenerative disorder\'], [\'disease\': \'loss of memory\'], [\'disease\': \'decline in cognitive abilities\']]',
 "gene - [['gene': 'APOE'], ['gene': 'ACKR2'], ['gene': 'ATP8B4'], ['gene': 'LAMB2'], ['gene': 'TOMM40']]",
 "proteinmutation - [['proteinmutation': 'V41A'], ['proteinmutation': 'R176C'], ['proteinmutation': 'G395S'], ['proteinmutation': 'E987K'], ['proteinmutation': 'R239W']]",
 "Title: Analysis of Alzheimer's disease associated deleterious non-synonymous single nucleotide polymorphisms and their impacts on protein structure and function by performing in-silico methods. Abstract: Alzheimer's disease (AD) is a neurodegenerative disorder that is presented with a progressive loss of memory, a decline in cognitive abilities and multiple changes in behavior. Its pathogenicity has been linked to genetic factors in approximately 60-80% of the case

In [12]:
pubtator_info

{"Alzheimer's disease": {'label': 'disease', 'unique_id': 'MESH:D000544'},
 'AD': {'label': 'disease', 'unique_id': 'MESH:D000544'},
 'neurodegenerative disorder': {'label': 'disease',
  'unique_id': 'MESH:D019636'},
 'loss of memory': {'label': 'disease', 'unique_id': 'MESH:D008569'},
 'decline in cognitive abilities': {'label': 'disease',
  'unique_id': 'MESH:D003072'},
 'APOE': {'label': 'gene', 'unique_id': 'Gene ID:348'},
 'ACKR2': {'label': 'gene', 'unique_id': 'Gene ID:1238'},
 'V41A': {'label': 'proteinmutation',
  'unique_id': 'tmVar:p|SUB|V|41|A;HGVS:p.V41A;VariantGroup:3;CorrespondingGene:1238;RS#:2228467;CorrespondingSpecies:9606;CA#:2338634'},
 'R176C': {'label': 'proteinmutation',
  'unique_id': 'tmVar:p|SUB|R|176|C;HGVS:p.R176C;VariantGroup:0;CorrespondingGene:348;RS#:7412;CorrespondingSpecies:9606;CA#:127498'},
 'ATP8B4': {'label': 'gene', 'unique_id': 'Gene ID:79895'},
 'G395S': {'label': 'proteinmutation',
  'unique_id': 'tmVar:p|SUB|G|395|S;HGVS:p.G395S;VariantGroup:

In [13]:
properties_info

{'proteinmutation': 'R239W', 'source': 'PMID39589570'}

In [ ]:
# -----------------Build the graph-----------------
itext2kg = iText2KG(llm_model = llm, embeddings_model = embeddings)
kg = itext2kg.build_graph(
    sections=[semantic_blocks], 
    source=properties_info,  
    entities_info=pubtator_info,
    ent_threshold=0.9, 
    rel_threshold=0.4
    )

2025-02-28 10:30:57,726 - INFO - Extracting Entities from the Document


2025-02-28 10:30:58,800 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-02-28 10:31:26,396 - INFO - INFO --- alzheimer's disease add unique_id and label
2025-02-28 10:31:26,397 - INFO - INFO --- apoe add unique_id and label
2025-02-28 10:31:26,398 - INFO - INFO --- ackr2 add unique_id and label
2025-02-28 10:31:26,399 - INFO - INFO --- atp8b4 add unique_id and label
2025-02-28 10:31:26,400 - INFO - INFO --- lamb2 add unique_id and label
2025-02-28 10:31:26,401 - INFO - INFO --- tomm40 add unique_id and label
2025-02-28 10:31:26,401 - INFO - INFO --- v41a add unique_id and label
2025-02-28 10:31:26,402 - INFO - INFO --- r176c add unique_id and label
2025-02-28 10:31:26,403 - INFO - INFO --- g395s add unique_id and label
2025-02-28 10:31:26,403 - INFO - INFO --- e987k add unique_id and label
2025-02-28 10:31:26,404 - INFO - INFO --- r239w add unique_id and label
2025-02-28 10:31:26,404 - INFO - {'entities': [{'label': 'disease', 'name': "Alzheimer's dis

In [ ]:
# kg.relationships[0]

In [ ]:
GraphIntegrator(uri=URI, username=USERNAME, password=PASSWORD).visualize_graph(knowledge_graph=kg)

2025-02-28 10:27:43,655 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (m))} {position: line: 1, column: 1, offset: 0} for query: 'MATCH (n:gene {name: "ackr2"}), (m:disease {name: "alzheimer\'s disease"}) MERGE (n)-[r:associated_with]->(m) SET r.embeddings = "0.00097569043,-0.0033646754,-0.15813373,-0.021601088,0.0036009245,-0.008725069,-0.013594078,-0.038311917,-0.0049836556